In [1]:
import numpy as np
import pandas as pd
import requests
import json
import re
from functools import reduce

import dateutil
from dateutil.parser import parse
import datetime
from sklearn import preprocessing
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# ESIF *Main Real Power Total* and *Main Energy Net*
start_date = '2017-03-01'
end_date = '2018-12-29'
start_time = '12:00:00'
end_time = '23:00:00'

In [12]:
root_url = 'https://internal-apis.nrel.gov/intelligentcampus/hisRead?id='
reference_id = ['@p:stm_campus:r:23752630-794c8dba','@p:stm_campus:r:1faa61e0-fca6c387']
feat_name = ['ESIF_Energy_Net','ESIF_Real_Power_Total']
start_date = ['2018-04-01','2018-05-01']
end_date = ['2018-04-30','2018-05-31']

response_dict = {}
for i in range(len(reference_id)):
    response_dict['resp_'+feat_name[i]] = {}
    print(response_dict.keys())
    for j in range(len(start_date)):
        date_range = '&range=\"'+start_date[j]+'%2c'+end_date[j]+'\"'
        url_temp = root_url+reference_id[i]+date_range
        print(url_temp)
        response_dict['resp_'+feat_name[i]]['d-range_'+str(j)] = requests.get(url_temp)
        if response_dict['resp_'+feat_name[i]]['d-range_'+str(j)].status_code == 200:
            pass
        else:
            print("response from {} is not getting fetched from API".format(feat_name[i]))
            print("the reponse code of failed request is: {}".format(response_dict['resp_'+feat_name[i]]['d-range_'+str(j)].status_code))
            pass

dict_keys(['resp_ESIF_Energy_Net'])
https://internal-apis.nrel.gov/intelligentcampus/hisRead?id=@p:stm_campus:r:23752630-794c8dba&range="2018-04-01%2c2018-04-30"
https://internal-apis.nrel.gov/intelligentcampus/hisRead?id=@p:stm_campus:r:23752630-794c8dba&range="2018-05-01%2c2018-05-31"
dict_keys(['resp_ESIF_Real_Power_Total', 'resp_ESIF_Energy_Net'])
https://internal-apis.nrel.gov/intelligentcampus/hisRead?id=@p:stm_campus:r:1faa61e0-fca6c387&range="2018-04-01%2c2018-04-30"
https://internal-apis.nrel.gov/intelligentcampus/hisRead?id=@p:stm_campus:r:1faa61e0-fca6c387&range="2018-05-01%2c2018-05-31"


In [18]:
#pd.concat({k: pd.DataFrame(v) for k,v in response_dict.items()})

In [19]:
def str_split(row):
    time_val = row.split(",")[0].strip(" Denver")
    energy_val = row.split(",")[1]
    energy_val = re.sub('[kwh%RHmbar°FW/m²_irrp]','', energy_val)
    return (time_val, float(energy_val))

def date_parser(row):
    parsed = parse(row)
    datetime_var = parsed.strftime(format='%m-%d-%y %H:%M:%S')
    date = parsed.date()
    time = parsed.time()
    return (datetime_var ,date, time)

In [40]:
feat_name = ['ESIF_Energy_Net','ESIF_Real_Power_Total']
start_date = ['2018-04-01','2018-05-01']
end_date = ['2018-04-30','2018-05-31']

parsed_dict = {}
for i in range(len(feat_name)):
    parsed_dict[feat_name[i]] = {}
    parsed_dict[feat_name[i]+'_dt_parsed'] = {}
    for j in range(len(start_date)):
        parsed_dict[feat_name[i]]['d-range_'+str(j)] = {}
        parsed_dict[feat_name[i]]['d-range_'+str(j)] = response_dict['resp_'+feat_name[i]]['d-range_'+str(j)].content.decode('utf-8').split("\n")
        parsed_dict[feat_name[i]]['d-range_'+str(j)] = parsed_dict[feat_name[i]]['d-range_'+str(j)][2:]
        parsed_dict[feat_name[i]]['d-range_'+str(j)] = filter(None, parsed_dict[feat_name[i]]['d-range_'+str(j)])
        parsed_dict[feat_name[i]]['d-range_'+str(j)] = list(map(str_split, parsed_dict[feat_name[i]]['d-range_'+str(j)]))
        print(j)
        parsed_dict[feat_name[i]]['d-range_'+str(j)] = list(zip(*parsed_dict[feat_name[i]]['d-range_'+str(j)]))
        
        
        parsed_dict[feat_name[i]+'_dt_parsed']['d-range_'+str(j)] = list(map(date_parser, parsed_dict[feat_name[i]]['d-range_'+str(j)][0]))
        parsed_dict[feat_name[i]+'_dt_parsed']['d-range_'+str(j)] = list(zip(*parsed_dict[feat_name[i]+'_dt_parsed']['d-range_'+str(j)]))

0
1
0
1
